In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

import math

In [2]:
data=pd.read_csv('~\Downloads\daily_data.csv')
data

,datetime,high,low,close,open,volume
0,2018-01-01,13818.55,12750.00,13135.00,13715.65,6970.441076
1,2018-01-02,14750.00,12890.02,14580.00,13135.00,16359.347340
2,2018-01-03,15473.49,14150.00,14749.97,14576.87,19457.238601
3,2018-01-04,15280.00,13918.04,14811.00,14710.00,18783.876561
4,2018-01-05,17118.13,14600.00,16579.00,14805.01,22383.657333
...,...,...,...,...,...,...
1468,2022-01-08,42246.37,40501.00,40679.82,41707.04,27031.900870
1469,2022-01-09,42300.00,40520.00,41966.99,40679.82,27291.009660
1470,2022-01-10,42786.70,39650.00,41604.58,41966.99,51064.904360
1471,2022-01-11,43100.00,41158.99,42755.00,41604.58,38062.818280


In [3]:
def get_macd(price,slow,fast,smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1-exp2).rename(columns = {'close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    frames =  [macd, signal]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

macd=get_macd(data.close,26,12,9)
macd

,macd,signal
0,0.000000,0.000000
1,115.270655,23.054131
2,217.827657,62.008836
3,300.564708,109.720011
4,502.998982,188.375805
...,...,...
1468,-2136.672368,-1568.284564
1469,-2233.642180,-1701.356087
1470,-2313.071268,-1823.699123
1471,-2257.170852,-1910.393469


In [198]:
def generate_signal_for_macd(df,data):
    datetime=[]
    signal=[]
    current=0
    datetime.append(data.datetime[0])
    signal.append(0)
    for i in range(1,len(df.macd)):
        if(df.macd[i]>=df.signal[i] ):
            datetime.append(data.datetime[i])
            signal.append(1)
        elif(df.macd[i]<=df.signal[i]):
            datetime.append(data.datetime[i])
            signal.append(-1)
        else:
            datetime.append(data.datetime[i])
            signal.append(0)
    a=pd.DataFrame(columns=['datetime','signals'])
    a['datetime']=datetime
    a['signals']=signal
    return a
            
        
        
output=generate_signal_for_macd(macd,data)
output

,datetime,signals
0,2018-01-01,0
1,2018-01-02,1
2,2018-01-03,1
3,2018-01-04,1
4,2018-01-05,1
...,...,...
1468,2022-01-08,-1
1469,2022-01-09,-1
1470,2022-01-10,-1
1471,2022-01-11,-1


In [199]:
def to_submit(data,output):
    datetime=[]
    trade=[]
    signal=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        if(i==(len(data.close)-1)):
            if(current==1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(-1)
            elif(current==-1):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
            else:
                break
            break
        if(output.signals[i]==1):
            if(current==1):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
            elif(current==0):
                datetime.append(output.datetime[i])
#                 price.append(data.close[i])
                trade.append('close')
                signal.append(1)
                current=1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(1)
                current=0
            
        elif(output.signals[i]==-1):
            if(current==1):
                datetime.append(output.datetime[i])
                signal.append(-1)
#                 price.append(data.close[i])
                trade.append('close')
                current=0
            elif(current==0):
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(-1)
                current=-1
            else:
                datetime.append(output.datetime[i])
                trade.append('close')
#                 price.append(data.close[i])
                signal.append(0)
                current=-1
        
        else:
            datetime.append(output.datetime[i])
            trade.append('close')
#             price.append(data.close[i])
            signal.append(0)
        
            
            
    to_submit=pd.DataFrame(columns=['datetime'])
    to_submit['datetime']=datetime
    to_submit['signals']=signal
    to_submit['open']=data.open
    
    to_submit['high']=data.high
    to_submit['low']=data.low
    to_submit['close']=data.close
    to_submit['volume']=data.volume
    
#     to_submit['close\open']=trade
#     to_submit['price']=price
    

    return to_submit
            
            

In [208]:
def trailing_stop_loss(data,submit):
    datetime=[]
    trade=[]
    signal=[]
    percent=0.05
    hello=0
    stop_loss=0
    current=0
    l=-1
    for i in range(len(submit.datetime)):
        datetime.append(submit.datetime[i])
        trade.append('close')
        if(i<=l):
            continue
        if(hello==0):
            if(submit.signals[i]==0):
                signal.append(0)
            elif(submit.signals[i]==1):
                hello=1
                current=data.close[i]
                stop_loss=((1-percent)*current)
                signal.append(1)
                
            else:
                hello=-1
                current=data.close[i]
                stop_loss=((1+percent)*current)
                signal.append(-1)          
                
        elif(hello==1):
            if(submit.signals[i]==-1):

                    hello=0
                    signal.append(-1)
                    stop_loss=0
                    
            elif(submit.signals[i]==0):
                if(data.close[i]>current):
                    current=data.close[i]
                    stop_loss=((1-percent)*current)
                    signal.append(0)
            
                    
                elif(data.close[i]<=stop_loss):
                    signal.append(-1)
                    stop_loss=0
                    hello=0
                    for j in range(i+1,len(submit.datetime)):
                        if(submit.signals[j]==-1):
                            signal.append(0)
                            l=j
                            break
                        signal.append(0)
                
                else:
                    signal.append(0)
                        
            
            
        else:#hello=-1
            if(submit.signals[i]==1):
                    signal.append(1)
                    stop_loss=0
                    hello=0
                    
            elif(submit.signals[i]==0):
                if(data.close[i]<current):
                    current=data.close[i]
                    stop_loss=((1+percent)*data.close[i])
                    signal.append(0)
                
                    
                elif(data.close[i]>=stop_loss):
                    signal.append(1)
                    stop_loss=0
                    hello=0
                    for j in range(i+1,len(submit.datetime)):
                        if(submit.signals[j]==1):
                            signal.append(0)
                            l=j
                            break
                        signal.append(0)   
                        
                else:
                    signal.append(0)
                
                
            
        
#     return signal   
    to_submit=pd.DataFrame(columns=['datetime'])
    to_submit['datetime']=datetime
    to_submit['signals']=signal
    to_submit['open']=data.open
    
    to_submit['high']=data.high
    to_submit['low']=data.low
    to_submit['close']=data.close
    to_submit['volume']=data.volume
    
#     to_submit['close\open']=trade
#     to_submit['price']=price
    

    return to_submit 
            
            
    

In [209]:

def only_long_then_short(data,output):
    datetime=[]
    signal=[]
    trade=[]
    price=[]
    current=0
    for i in range(len(data.close)):
        datetime.append(data.datetime[i])
        trade.append('close')
#         price.append(data.close[i])
        if(i==len(data.close)-1):
            if(current==1):
                signal.append(-1)
                break
            else:
                signal.append(0)
                break
            
        
        if(output.signals[i]==1):
            if(current==0):
                signal.append(1)
                current=1
            else:
                signal.append(0)
        elif(output.signals[i]==-1):
            if(current==1):
                signal.append(-1)
                current=0
            else:
                signal.append(0)
            
        else:
            if(current==0):
                signal.append(0)
            else:
                signal.append(0)
    a=pd.DataFrame(columns=['datetime'])
    a['datetime']=datetime
    a['signals']=signal
    a['open']=data.open
    a['high']=data.high
    a['low']=data.low
    a['close']=data.close
    a['volume']=data.volume
#     a['close\open']=trade
#     to_submit['price']=price
    
    
    
    return a
            

In [210]:
#static approach
def dd_long_short(trade_type,close):
    
    if trade_type == 'Long then short':
        cumulative_max_values = close.cummax()
        cumulative_returns = close/ cumulative_max_values - 1
        max_drawdown = cumulative_returns.min()
        return max_drawdown
        
    elif trade_type == 'Short then long':
        pip=[]
        for i in close:
            pip.append(i)
        stock = -(1000 /pip[0])
        portfolio = []

        for i in range(len(pip)):
            profit = stock * (pip[i] - pip[0])
            portfolio.append(1000+profit)
        
        drawdown = []
        new = 0

        for i in range(len(portfolio)):
            new = max(new,portfolio[i])
            drawdown.append(((new-portfolio[i])/new))
            
        return np.max(drawdown)




def metrics1(data,output):
    PL=[]
    PL_in_percent=[]
    hello=0
    number_of_stock=[]
    capital=1000
    entry=[]
    exit=[]
    exit_index=[]
    entry_index=[]
    holding_duration=[]
    drawdown=[]
    dip=[]
    
    commision_rate=1
    risk_free_rate=0.05
    
    for i in range(len(data.close)):
        if(output.signals[i]==1):
            if(hello==0):
                hello=1
                entry_index.append(i)
                entry.append(data.datetime[i])
                
                number_of_stock=(capital)/(data.close[i])
                capital=0
            else:
                hello=0
                exit_index.append(i)
                exit.append(data.datetime[i])
                holding_duration.append(exit_index[-1]-entry_index[-1])
                capital=(capital-(number_of_stock)*(data.close[i]))
                x=entry_index[-1]
#                 if x in data.close.index and i+1 in data.close.index:
                drawdown.append(dd_long_short('Short then long',data.close[x:i+1]))
                dip.append((np.max(data.close[x:i+1])*number_of_stock-1000)/10)    
    
                number_of_stock=0
                PL.append(capital-1000)
                PL_in_percent.append((capital-1000)/10)
                capital=1000
                
        elif(output.signals[i]==-1):
            if(hello==0):
                hello=-1
                
                entry_index.append(i)
                entry.append(data.datetime[i])
                number_of_stock=(capital/(data.close[i]))
                capital=capital+(number_of_stock)*(data.close[i])
            else:
                hello=0
                exit_index.append(i)
                exit.append(data.datetime[i])
                holding_duration.append(exit_index[-1]-entry_index[-1])
                capital=capital+(number_of_stock)*(data.close[i])
                x=entry_index[-1]
                drawdown.append(dd_long_short('Long then short',(data.close[x:i+1])))
                dip.append((1000-(np.min(data.close[x:i+1])*number_of_stock))/10)    
                number_of_stock=0
                PL.append(capital-1000)
                PL_in_percent.append((capital-1000)/10)
                capital=1000
                
                
        
    index=[]            
    for i in range(len(PL)):
        index.append(i)
    a=pd.DataFrame(columns=['trade number','entry','exit','holding duration','return','return %'])
    
    a['trade number']=index
    a['entry']=entry
    a['exit']=exit
    a['holding duration']=holding_duration
    a['return']=PL
    a['return %']=PL_in_percent
    a['drawdown']=drawdown
    a['dip']=dip           
    profit=[]
    loss=[]
    for i in range(len(PL)):
        if(PL[i]>0):
            profit.append(PL[i])
        else:
            loss.append(PL[i])
    print("Total Closed trades: ",len(PL))
    print("Winning trades: ",(len(profit)))
    print("Losing Trades: ",(len(PL)-len(profit)))
    number_of_stock=(1000/(data.close[0]))
    bench_mark_return=(number_of_stock)*(data.close[len(data.close)-1])
    bench_mark_return=(bench_mark_return-1000)
    print("Benchmark Return: ",bench_mark_return)
    print("WIN rate: ",(100*len(profit))/len(PL) , " %")
    print("Gross Profit: ",np.sum(PL))
    net_profit=(np.sum(PL)-(len(PL)*(commision_rate)))
    print("Net Profit: ",net_profit)
    print("Average Profit: ",net_profit/(len(PL)))
    print("MAX Drawdowns: ", (100*(np.min(drawdown)))," %")
    print("Largest Win: ", np.max(profit))
    print("Average Win: ",(np.mean(profit)))
    
   
    print("Largest Loss: ",np.min(loss))
    print("Average Loss: ",np.mean(loss))
    print("Maximum Holding Time: ",np.max(holding_duration))
    print("Average Holding Duration: ",((np.sum(holding_duration))/(len(holding_duration))))
    print("Maximum Dip: ",np.max(dip))
    print("Avg. Dip: ",np.mean(dip))
    print("Sharpe Ratio: ",((np.mean(PL)-risk_free_rate)/(np.std(PL))))
    average_return = np.mean(PL)
    downside_returns = [ret for ret in PL if ret < risk_free_rate]
    downside_volatility = np.std(downside_returns)
    
    sortino_ratio = (average_return - risk_free_rate) / downside_volatility
    print("Sortino Ratio: ", sortino_ratio )
    
   
    print("returns: ",((np.sum(PL)/(10*len(PL))))," %")
    
    return a         
            
    

In [211]:
a=metrics1(data,to_submit(data,output))

Total Closed trades:  90
Winning trades:  39
Losing Trades:  51
Benchmark Return:  2248.9531785306435
WIN rate:  43.333333333333336  %
Gross Profit:  3573.3856537157108
Net Profit:  3483.3856537157108
Average Profit:  38.70428504128567
MAX Drawdowns:  -25.876296889191885  %
Largest Win:  536.2987182514441
Average Win:  177.6964675330233
Largest Loss:  -226.71593602848884
Average Loss:  -65.81914862886664
Maximum Holding Time:  71
Average Holding Duration:  15.355555555555556
Maximum Dip:  22.671593602848883
Avg. Dip:  4.867685335043812
Sharpe Ratio:  0.25347078121347333
Sortino Ratio:  0.8733915339919821
returns:  3.9704285041285674  %


In [212]:
a

,trade number,entry,exit,holding duration,return,return %,drawdown,dip
0,0,2018-01-02,2018-01-11,9,-62.997257,-6.299726,-0.185355,6.299726e+00
1,1,2018-01-12,2018-01-29,17,151.153874,15.115387,0.197255,6.371295e+00
2,2,2018-01-30,2018-02-11,12,174.756472,17.475647,0.125885,1.136868e-14
3,3,2018-02-12,2018-03-08,24,73.968019,7.396802,-0.202152,1.330418e+00
4,4,2018-03-09,2018-03-23,14,30.648864,3.064886,0.139377,7.806966e+00
...,...,...,...,...,...,...,...,...
85,85,2021-10-25,2021-11-09,15,-53.758826,-5.375883,0.115374,5.375883e+00
86,86,2021-11-10,2021-11-12,2,-57.829879,-5.782988,-0.057830,5.782988e+00
87,87,2021-11-13,2021-12-19,36,265.271727,26.527173,0.061349,0.000000e+00
88,88,2021-12-20,2022-01-04,15,-11.208038,-1.120804,-0.114928,1.120804e+00


In [213]:
daily_metrics=metrics1(data,trailing_stop_loss(data,to_submit(data,output)))

Total Closed trades:  90
Winning trades:  38
Losing Trades:  52
Benchmark Return:  2248.9531785306435
WIN rate:  42.22222222222222  %
Gross Profit:  1387.263009097085
Net Profit:  1297.263009097085
Average Profit:  14.414033434412055
MAX Drawdowns:  -19.703510653063294  %
Largest Win:  340.42142306304413
Average Win:  108.96727254803238
Largest Loss:  -115.86012128743687
Average Loss:  -52.95179514861817
Maximum Holding Time:  29
Average Holding Duration:  7.288888888888889
Maximum Dip:  11.586012128743686
Avg. Dip:  3.272412037993537
Sharpe Ratio:  0.14584864444221382
Sortino Ratio:  0.5376622807918214
returns:  1.5414033434412056  %


In [215]:
c=metrics1(data,only_long_then_short(data,output))

Total Closed trades:  47
Winning trades:  19
Losing Trades:  28
Benchmark Return:  2248.9531785306435
WIN rate:  40.42553191489362  %
Gross Profit:  2851.0013184889476
Net Profit:  2804.0013184889476
Average Profit:  59.659602521041435
MAX Drawdowns:  -25.876296889191885  %
Largest Win:  735.1464286772573
Average Win:  243.2910804029299
Largest Loss:  -217.96301012440472
Average Loss:  -63.26890032738288
Maximum Holding Time:  72
Average Holding Duration:  16.340425531914892
Maximum Dip:  21.796301012440473
Avg. Dip:  4.450970098546247
Sharpe Ratio:  0.309859636602874
Sortino Ratio:  1.3163902782588712
returns:  6.065960252104144  %


In [216]:
c

,trade number,entry,exit,holding duration,return,return %,drawdown,dip
0,0,2018-01-02,2018-01-11,9,-62.997257,-6.299726,-0.185355,6.299726
1,1,2018-01-29,2018-01-30,1,-114.704957,-11.470496,-0.114705,11.470496
2,2,2018-02-11,2018-03-08,25,122.013682,12.201368,-0.202152,0.000000
3,3,2018-03-23,2018-03-27,4,-87.218642,-8.721864,-0.117317,8.721864
4,4,2018-04-08,2018-05-08,30,316.716167,31.671617,-0.067929,3.571148
5,5,2018-06-03,2018-06-11,8,-122.125554,-12.212555,-0.124315,12.212555
6,6,2018-06-20,2018-06-23,3,-94.339623,-9.433962,-0.094340,9.433962
7,7,2018-06-28,2018-06-29,1,-36.624386,-3.662439,-0.036624,3.662439
8,8,2018-06-30,2018-08-01,32,194.091050,19.409105,-0.087883,2.335622
9,9,2018-08-21,2018-09-07,17,-5.099225,-0.509922,-0.127877,0.647132


In [217]:
d=metrics1(data,trailing_stop_loss(data,only_long_then_short(data,output)))

Total Closed trades:  47
Winning trades:  19
Losing Trades:  28
Benchmark Return:  2248.9531785306435
WIN rate:  40.42553191489362  %
Gross Profit:  1549.2015262816944
Net Profit:  1502.2015262816944
Average Profit:  31.96173460173818
MAX Drawdowns:  -19.703510653063294  %
Largest Win:  513.9161528413531
Average Win:  151.3934360843991
Largest Loss:  -119.52852305287831
Average Loss:  -47.402634261496004
Maximum Holding Time:  21
Average Holding Duration:  8.340425531914894
Maximum Dip:  11.95285230528783
Avg. Dip:  3.0718223127435986
Sharpe Ratio:  0.24903333835465224
Sortino Ratio:  1.0107930278757977
returns:  3.296173460173818  %


In [218]:
btc=yf.download("BTC-USD",'2022-01-12','2022-12-31')
btc['datetime']=btc.index
btc

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,datetime
Date,,,,,,,
2022-01-12,42742.179688,44135.367188,42528.988281,43949.101562,43949.101562,33499938689,2022-01-12
2022-01-13,43946.742188,44278.421875,42447.042969,42591.570312,42591.570312,47691135082,2022-01-13
2022-01-14,42598.871094,43346.687500,41982.617188,43099.699219,43099.699219,23577403399,2022-01-14
2022-01-15,43101.898438,43724.671875,42669.035156,43177.398438,43177.398438,18371348298,2022-01-15
2022-01-16,43172.039062,43436.808594,42691.023438,43113.878906,43113.878906,17902097845,2022-01-16
...,...,...,...,...,...,...,...
2022-12-26,16842.250000,16920.123047,16812.369141,16919.804688,16919.804688,11886957804,2022-12-26
2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,15748580239,2022-12-27
2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,17005713920,2022-12-28


In [173]:
tether=yf.download("USDT-USD",'2022-01-12','2022-12-31')
tether

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-12,1.000369,1.000713,1.000210,1.000290,1.000290,62300703831
2022-01-13,1.000287,1.000763,1.000262,1.000405,1.000405,63231903910
2022-01-14,1.000411,1.000757,1.000162,1.000283,1.000283,60503324000
2022-01-15,1.000301,1.000940,1.000222,1.000642,1.000642,41922988280
2022-01-16,1.000727,1.000818,1.000243,1.000417,1.000417,41551382127
...,...,...,...,...,...,...
2022-12-26,0.999934,1.000020,0.999894,0.999995,0.999995,14986560291
2022-12-27,0.999964,1.000019,0.999828,0.999850,0.999850,18554435182
2022-12-28,0.999855,0.999921,0.999652,0.999686,0.999686,21161758512


In [219]:
close_values=[]
for i in range(len(tether.Close)):
    close_values.append(btc.Close[i]/tether.Close[i])
close_values

[43936.35842799481,
 42574.32966925067,
 43087.505331795495,
 43149.698832486516,
 43095.90816703221,
 42235.00833717145,
 42357.84360563485,
 41723.88549978684,
 40669.96516836846,
 36450.86366313222,
 35024.71776496904,
 36259.00068189994,
 36638.24218488134,
 36931.69592580127,
 36828.9180984318,
 37125.4618050135,
 37771.30124642763,
 38124.37719088451,
 37904.67387867369,
 38464.471908602216,
 38726.77374060763,
 36938.94612690776,
 37140.265431944295,
 41477.52209843744,
 41416.52232659673,
 42383.65440946623,
 43815.96561051815,
 44091.15051643469,
 44305.83399129633,
 43533.50862556342,
 42377.338914179425,
 42219.09282755417,
 42178.283359783745,
 42565.08388662765,
 44549.14365864599,
 43944.4148953436,
 40517.14603151606,
 40008.61285380465,
 40099.058418955756,
 38415.70324799801,
 37056.67915583548,
 38259.091028259645,
 37278.0037637085,
 38308.24603693401,
 39195.640951582965,
 39089.63114646462,
 37691.27786951726,
 43178.08093473055,
 44334.595636051665,
 43903.6167103

In [220]:
btc['close']=close_values
btc['open']=btc['Open']
btc['high']=btc['High']
btc['low']=btc['Low']
btc['volume']=btc['Volume']
btc

,Open,High,Low,Close,Adj Close,Volume,datetime,close,open,high,low,volume
Date,,,,,,,,,,,,
2022-01-12,42742.179688,44135.367188,42528.988281,43949.101562,43949.101562,33499938689,2022-01-12,43936.358428,42742.179688,44135.367188,42528.988281,33499938689
2022-01-13,43946.742188,44278.421875,42447.042969,42591.570312,42591.570312,47691135082,2022-01-13,42574.329669,43946.742188,44278.421875,42447.042969,47691135082
2022-01-14,42598.871094,43346.687500,41982.617188,43099.699219,43099.699219,23577403399,2022-01-14,43087.505332,42598.871094,43346.687500,41982.617188,23577403399
2022-01-15,43101.898438,43724.671875,42669.035156,43177.398438,43177.398438,18371348298,2022-01-15,43149.698832,43101.898438,43724.671875,42669.035156,18371348298
2022-01-16,43172.039062,43436.808594,42691.023438,43113.878906,43113.878906,17902097845,2022-01-16,43095.908167,43172.039062,43436.808594,42691.023438,17902097845
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,16842.250000,16920.123047,16812.369141,16919.804688,16919.804688,11886957804,2022-12-26,16919.889402,16842.250000,16920.123047,16812.369141,11886957804
2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,15748580239,2022-12-27,16719.682197,16919.291016,16959.845703,16642.072266,15748580239
2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,17005713920,2022-12-28,16557.771361,16716.400391,16768.169922,16497.556641,17005713920


In [221]:
macd1=get_macd(btc.close,26,12,9)
macd1

,macd,signal
Date,,
2022-01-12,0.000000,0.000000
2022-01-13,-108.651867,-21.730373
2022-01-14,-151.602685,-47.704836
2022-01-15,-178.564641,-73.876797
2022-01-16,-201.944749,-99.490387
...,...,...
2022-12-26,-97.317752,-104.098084
2022-12-27,-103.680903,-104.014648
2022-12-28,-120.400693,-107.291857


In [222]:
output1=generate_signal_for_macd(macd1,btc)
output1

,datetime,signals
0,2022-01-12,0
1,2022-01-13,-1
2,2022-01-14,-1
3,2022-01-15,-1
4,2022-01-16,-1
...,...,...
348,2022-12-26,1
349,2022-12-27,1
350,2022-12-28,-1
351,2022-12-29,-1


In [223]:
to_submit(btc,output1)

,datetime,signals,open,high,low,close,volume
0,2022-01-12,0,NaN,NaN,NaN,NaN,NaN
1,2022-01-13,-1,NaN,NaN,NaN,NaN,NaN
2,2022-01-14,0,NaN,NaN,NaN,NaN,NaN
3,2022-01-15,0,NaN,NaN,NaN,NaN,NaN
4,2022-01-16,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
348,2022-12-26,1,NaN,NaN,NaN,NaN,NaN
349,2022-12-27,1,NaN,NaN,NaN,NaN,NaN
350,2022-12-28,-1,NaN,NaN,NaN,NaN,NaN
351,2022-12-29,-1,NaN,NaN,NaN,NaN,NaN


In [224]:
metrics1(btc,to_submit(btc,output1))

Total Closed trades:  24
Winning trades:  11
Losing Trades:  13
Benchmark Return:  -621.9930391502658
WIN rate:  45.833333333333336  %
Gross Profit:  105.68909736152034
Net Profit:  81.68909736152034
Average Profit:  3.4037123900633475
MAX Drawdowns:  -15.794423706910543  %
Largest Win:  200.68805813007134
Average Win:  68.63220712474386
Largest Loss:  -133.61990084325043
Average Loss:  -49.94347546235862
Maximum Holding Time:  41
Average Holding Duration:  13.666666666666666
Maximum Dip:  13.361990084325043
Avg. Dip:  3.7846126167680367
Sharpe Ratio:  0.056892318414100744
Sortino Ratio:  0.10600119545099482
returns:  0.44037123900633474  %


,trade number,entry,exit,holding duration,return,return %,drawdown,dip
0,0,2022-01-13,2022-01-31,18,96.533705,9.653370,0.068625,1.351446e+00
1,1,2022-02-01,2022-02-18,17,33.099559,3.309956,-0.101922,4.616516e+00
2,2,2022-02-19,2022-02-28,9,-76.785407,-7.678541,0.141891,7.678541e+00
3,3,2022-03-01,2022-03-06,5,-133.619901,-13.361990,-0.133620,1.336199e+01
4,4,2022-03-07,2022-03-16,9,-80.633108,-8.063311,0.102825,1.028245e+01
5,5,2022-03-17,2022-04-05,19,112.781335,11.278133,-0.040459,1.136868e-14
6,6,2022-04-06,2022-05-04,28,81.092094,8.109209,0.042233,6.768701e-01
7,7,2022-05-05,2022-05-20,15,200.688058,20.068806,0.051661,0.000000e+00
8,8,2022-05-21,2022-06-12,22,-90.963760,-9.096376,-0.157944,9.096376e+00
9,9,2022-06-13,2022-06-25,12,44.717241,4.471724,0.094981,3.450236e-01


In [196]:
metrics1(btc,to_submit(btc,output1))

Total Closed trades:  24
Winning trades:  11
Losing Trades:  13
Benchmark Return:  -621.9930391502658
WIN rate:  45.833333333333336  %
Gross Profit:  105.68909736152034
Net Profit:  81.68909736152034
Average Profit:  3.4037123900633475
MAX Drawdowns:  -15.794423706910543  %
Largest Win:  200.68805813007134
Average Win:  68.63220712474386
Largest Loss:  -133.61990084325043
Average Loss:  -49.94347546235862
Maximum Holding Time:  41
Average Holding Duration:  13.666666666666666
Maximum Dip:  13.361990084325043
Avg. Dip:  3.7846126167680367
Sharpe Ratio:  0.056892318414100744
Sortino Ratio:  0.10600119545099482
returns:  0.44037123900633474  %


,trade number,entry,exit,holding duration,return,return %,drawdown,dip
0,0,2022-01-13,2022-01-31,18,96.533705,9.653370,0.068625,1.351446e+00
1,1,2022-02-01,2022-02-18,17,33.099559,3.309956,-0.101922,4.616516e+00
2,2,2022-02-19,2022-02-28,9,-76.785407,-7.678541,0.141891,7.678541e+00
3,3,2022-03-01,2022-03-06,5,-133.619901,-13.361990,-0.133620,1.336199e+01
4,4,2022-03-07,2022-03-16,9,-80.633108,-8.063311,0.102825,1.028245e+01
5,5,2022-03-17,2022-04-05,19,112.781335,11.278133,-0.040459,1.136868e-14
6,6,2022-04-06,2022-05-04,28,81.092094,8.109209,0.042233,6.768701e-01
7,7,2022-05-05,2022-05-20,15,200.688058,20.068806,0.051661,0.000000e+00
8,8,2022-05-21,2022-06-12,22,-90.963760,-9.096376,-0.157944,9.096376e+00
9,9,2022-06-13,2022-06-25,12,44.717241,4.471724,0.094981,3.450236e-01
